In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import mlflow
import dagshub

# Initialize Dagshub with MLflow
dagshub.init(repo_owner='Janithadilshan10', repo_name='e19-co544-Diabetes-Health-Indicator', mlflow=True)

# Load the test dataa
X_test = pd.read_csv('/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/data/X_test.csv')
y_test = pd.read_csv('/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/data/y_test.csv').squeeze()  # Ensure y_test is a Series, not a DataFrame

# Load the training data for evaluation purposes
X_train = pd.read_csv('/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/data/X_train.csv')
y_train = pd.read_csv('/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/data/y_train.csv').squeeze()  # Ensure y_train is a Series, not a DataFrame

# List of models and their corresponding paths
models = {
    'Random Forest': '/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/models/random_forest.pkl',
    'Logistic Regression': '/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/models/logistic_regression.pkl',
    'Gradient Boosting': '/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/models/gradient_boosting.pkl',
    'SVM': '/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/models/svm_model.pkl'
}

# Dictionary to store evaluation results
results = {}

# Track the model with the highest accuracy
best_accuracy = 0
best_model_name = None
best_model = None

# Function to evaluate a model
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    with mlflow.start_run():
        # Training data evaluation
        y_train_pred = model.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_confusion_mat = confusion_matrix(y_train, y_train_pred)

        # Testing data evaluation
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        test_confusion_mat = confusion_matrix(y_test, y_pred)

        # Log metrics to MLflow
        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('training_accuracy', train_accuracy)

        # Compute mean squared error as an example (you can replace this with relevant metrics)
        mse = ((y_test - y_pred) ** 2).mean()
        mlflow.log_metric('mse', mse)

        # Log confusion matrices as artifacts
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        sns.heatmap(test_confusion_mat, annot=True, fmt="d", cmap="Blues")
        plt.title('Test Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.subplot(1, 2, 2)
        sns.heatmap(train_confusion_mat, annot=True, fmt="d", cmap="Blues")
        plt.title('Train Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.tight_layout()
        
        # Save confusion matrix plot as artifact
        confusion_matrix_plot_path = 'confusion_matrix_plot.png'
        plt.savefig(confusion_matrix_plot_path)
        mlflow.log_artifact(confusion_matrix_plot_path, 'confusion_matrices')

        # Log classification report as text artifact
        classification_report_text_path = 'classification_report.txt'
        with open(classification_report_text_path, 'w') as f:
            f.write(report)
        mlflow.log_artifact(classification_report_text_path, 'classification_reports')

        # Log model artifact
        model_path = 'model.pkl'
        with open(model_path, 'wb') as file:
            pickle.dump(model, file)
        mlflow.log_artifact(model_path)

        # Log parameters
        mlflow.log_param('model_name', model_name)

        # Log to Dagshub
        dagshub.log_hyperparams({
            'model_name': model_name,
            'train_accuracy': train_accuracy,
            'test_accuracy': accuracy,
            'mse': mse
        })
        dagshub.log_metrics({
            'accuracy': accuracy,
            'training_accuracy': train_accuracy,
            'mse': mse
        })

        return accuracy, report, test_confusion_mat, train_accuracy, train_confusion_mat

# Evaluate each model
for model_name, model_path in models.items():
    # Load the model from file
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    
    # Evaluate the model
    accuracy, report, test_confusion_mat, train_accuracy, train_confusion_mat = evaluate_model(model, X_train, y_train, X_test, y_test, model_name)
    
    # Store the results
    results[model_name] = {
        'accuracy': accuracy,
        'training set accuracy': train_accuracy,
        'report': report,
        'test_confusion_matrix': test_confusion_mat,
        'train_confusion_matrix': train_confusion_mat
    }

    # Check if this model has the best accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name
        best_model = model

# Save the best model as deploy.pkl
if best_model is not None:
    deploy_model_path = '/home/e19452/ml_project/e19-co544-Diabetes-Health-Indicator/models/deploy.pkl'
    with open(deploy_model_path, 'wb') as file:
        pickle.dump(best_model, file)
    mlflow.log_artifact(deploy_model_path)

# Print and visualize results for each model
# for model_name, result in results.items():
#     print(f"Model: {model_name}")
#     print(f"Testing data Accuracy: {result['accuracy']:.4f}")
#     print(f"Training data Accuracy: {result['training set accuracy']:.4f}")
#     print("Classification Report:")
#     print(result['report'])
#     print("Test Confusion Matrix:")
#     print(result['test_confusion_matrix'])
#     print("Train Confusion Matrix:")
#     print(result['train_confusion_matrix'])
#     print("\n")
    
#     # Visualize confusion matrices
#     fig, axes = plt.subplots(1, 2, figsize=(14, 5))

#     # Test Confusion Matrix
#     sns.heatmap(result['test_confusion_matrix'], annot=True, fmt="d", cmap="Blues", ax=axes[0])
#     axes[0].set_title(f'Confusion Matrix: {model_name} (Test Data)')
#     axes[0].set_xlabel('Predicted')
#     axes[0].set_ylabel('Actual')

#     # Train Confusion Matrix
#     sns.heatmap(result['train_confusion_matrix'], annot=True, fmt="d", cmap="Blues", ax=axes[1])
#     axes[1].set_title(f'Confusion Matrix: {model_name} (Train Data)')
#     axes[1].set_xlabel('Predicted')
#     axes[1].set_ylabel('Actual')

#     plt.tight_layout()
#     plt.show()
